Roosevelt University

https://www.roosevelt.edu/contact/directory?txtSearch=&ddlGroupSearch%5B%5D=1&ddlCollegeSearch=All&ddlInstrumentSearch=All&ddlDepartmentSearch=678

**bibliotecas**

In [6]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup

**Inicializando uma istancia do Google Chrome**

In [34]:
# Initialize function
service = Service()

# Set preferences for Chrome
options = webdriver.ChromeOptions()

# Start
driver = webdriver.Chrome(service=service, options=options)


In [35]:
url = "https://www.roosevelt.edu/contact/directory?txtSearch=&ddlGroupSearch%5B%5D=1&ddlCollegeSearch=All&ddlInstrumentSearch=All&ddlDepartmentSearch=678"
driver.get(url)

driver.implicitly_wait(10)  # Wait 10 seconds 

In [36]:
# Get the HTML of the page after it has loaded completely
html = driver.page_source

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(html, "html.parser")


**Encontrando os nomes dos professores com BS**

In [37]:
# Find all divs with class "indFacultyListing-infoContainer"
divs_professors = soup.find_all("div", class_="indFacultyListing-infoContainer")

# List to store the data of professors
data_professors = []

In [45]:
# List to store the data of professors
data_professors = []

# Iterate over the divs of professors
for div_professor in divs_professors:

    #NAME
    nome_professor = div_professor.find("div", class_="indDirectoryListing-name").find("span").get_text(strip=True)
    
    #LINK
    link_professor = div_professor.find("a", title="View {}'s Faculty Profile".format(nome_professor.strip())).get("href")
    
    # Get the full URL 
    full_link_professor = "https://www.roosevelt.edu" + link_professor
    
    # Open the professor's profile page with Selenium
    driver.get(full_link_professor)
    
    # Wait for the page to load completely
    driver.implicitly_wait(10)
    
    # Get the HTML of the profile page after it has loaded completely
    profile_html = driver.page_source
    
    # Parse the HTML using BeautifulSoup
    profile_soup = BeautifulSoup(profile_html, "html.parser")

    #EMAIL
    email_prefessor = profile_soup.find('div', class_='field_email').find('a').text.strip()

    #PHONE
    phone_element = profile_soup.find('div', class_='field_primary_phone')

    if phone_element:
        phone_professor = phone_element.find('a').text.strip()
    else:
        phone_professor = "NaN"

        
    data_professors.append({
        "Name": nome_professor,
        "Link": full_link_professor,
        "Email": email_prefessor,
        "Phone": phone_professor,
    })

# Create a Pandas DataFrame with the data of professors
df_professors = pd.DataFrame(data_professors)

# Print the DataFrame
print(df_professors)
   


                     Name                                               Link  \
0          Evgeny Dantsin         https://www.roosevelt.edu/profile/edantsin   
1  Saba Jamaliannasrabadi  https://www.roosevelt.edu/profile/sjamaliannas...   
2            Michael Ruth            https://www.roosevelt.edu/profile/mruth   
3           Rami Salahieh        https://www.roosevelt.edu/profile/msalahieh   
4            Alex Wolpert         https://www.roosevelt.edu/profile/awolpert   
5              Ray Wright          https://www.roosevelt.edu/profile/rwright   

                              Email         Phone  
0            edantsin@roosevelt.edu  312-281-3121  
1  sjamaliannasrabadi@roosevelt.edu           NaN  
2               mruth@roosevelt.edu  312-281-3303  
3           msalahieh@roosevelt.edu           NaN  
4            awolpert@roosevelt.edu  312-281-3120  
5             rwright@roosevelt.edu           NaN  


In [30]:
# Salvar o DataFrame em um arquivo CSV
df_professors.to_csv("df_professors.csv", index=False)


In [20]:
# Create a Pandas DataFrame with the data of professors
df_professors = pd.DataFrame(data_professors)

# Save the DataFrame directly to an Excel sheet
with pd.ExcelWriter('df_professors.xlsx', engine='xlsxwriter') as writer:
    df_professors.to_excel(writer, sheet_name='Sheet1', index=False)
